In [17]:
import numpy as np
from pole_gen.pole_gen import generate_pole, normalize_mesh
import open3d as o3d
from utils.plot import plot_open3d
import random
from utils.mesh_tools import create_quad

mesh = o3d.geometry.TriangleMesh()

# Simulate having a road at a given side (1, -1) or not (0)
# Index 0 = x axis, index 1 = z axis
road_presence = [random.choice([1, 0, -1]), random.choice([1, 0, -1])]
if all(road == 0 for road in road_presence):
    road_presence[random.choice([0, 1])] = random.choice([1, -1])
# Pick one of the roads to be the "main" road
main_road = random.choice([i for i, v in enumerate(road_presence) if v != 0])

# Useful for rotating things to the main road
rot_index = 0
match main_road:
    case 0:
        match road_presence[0]:
            case 1:
                rot_index = 0
            case -1:
                rot_index = 2
    case 1:
        match road_presence[1]:
            case 1:
                rot_index = 1
            case -1:
                rot_index = 3

# Create quad to represent road
road_meshes = []
if road_presence[0] != 0:
    road_meshes.append(
        create_quad(
            (road_presence[0], 0, 0),
            1 if main_road == 0 else 0.5,
            4 if main_road == 0 else 2,
        )
    )
if road_presence[1] != 0:
    road_meshes.append(
        create_quad(
            (0, road_presence[1], 0),
            4 if main_road == 1 else 2,
            1 if main_road == 1 else 0.5,
        )
    )

for road_mesh in road_meshes:
    mesh = mesh + road_mesh

# Generate pole
pole_mesh = generate_pole()
pole_mesh.scale(random.uniform(1.0, 1.955), center=(0, 0, 0))
mesh += pole_mesh

if random.random() <= 0.8:
    # Generate lamp
    lamp_mesh = o3d.io.read_triangle_mesh("pole_gen/meshes/lamp.ply")
    # Rotate to (main) road
    lamp_mesh.rotate(
        lamp_mesh.get_rotation_matrix_from_xyz((0, 0, np.deg2rad(90 * rot_index))),
        center=(0, 0, 0),
    )
    # Randomize rotation
    lamp_mesh.rotate(
        lamp_mesh.get_rotation_matrix_from_xyz(
            (0, np.deg2rad(random.uniform(-5.0, 0.0)), 0)
        ),
        center=(0, 0, 0),
    )
    # Randomize position
    lamp_mesh.translate(
        [0, 0, 7.5 + random.uniform(-1.0, 1.94)]
    )  # TODO Make sure this doesnt go higher than the pole
    mesh += lamp_mesh

# Normalize mesh
normalize_mesh(mesh)
plot_open3d([mesh])